# Test Tiny A11y LoRA Model
This notebook loads your LoRA fine-tuned `younglim/tiny-a11y-model` on DeepSeek-Coder and runs prompts.

In [ ]:
# Install required packages
!pip install --upgrade pip
!pip install transformers accelerate peft

In [ ]:
# Load base model and tokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModelForCausalLM

base_model_name = 'deepseek-ai/deepseek-coder-1.3b-instruct'
lora_model_name = 'younglim/tiny-a11y-model'

tokenizer = AutoTokenizer.from_pretrained(base_model_name)
base_model = AutoModelForCausalLM.from_pretrained(base_model_name)

# Load LoRA adapter using PeftModelForCausalLM
model = PeftModelForCausalLM.from_pretrained(base_model, lora_model_name)

In [ ]:
# Create text-generation pipeline
from transformers import pipeline

# Text-generation pipeline (do NOT set device if using accelerate)
generator = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=3000,
    do_sample=True,
    temperature=0.7,
    top_p=0.9
)

# Example prompt
prompt = 'Explain why alt text is important for images.'
output = generator(prompt, max_new_tokens=100)
print(output[0]['generated_text'])

In [ ]:
# Interactive prompt testing
custom_prompt = input('Enter your prompt: ')
result = generator(custom_prompt, max_new_tokens=100)
print(result[0]['generated_text'])